<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#compare performance of different selection methods, crossovers, and mutations

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [3]:
import sys
sys.path.append('..')

In [4]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations
from typing import Callable
#from library.algorithms.genetic_algorithms.algorithm import get_best_ind
#from library.algorithms.genetic_algorithms.mutation import inversion_mutation

In [5]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [6]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]])

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [7]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                #while any(idx in table for table in tables):
                    #idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self, verbose=False):
        total_happiness = 0
        i = 1
        for table in self.repr:
            # Sum relationship scores for all unique guest pairs at the table
            table_happiness = sum(self.relationship_matrix[a, b] for a, b in combinations(table, 2))
            total_happiness += table_happiness
            if verbose:
                print(f"Table {i} Happiness = {table_happiness}")
                i += 1
        return total_happiness

In [8]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness(verbose=True))

Random solution: [[3, 52, 40, 25, 16, 31, 35, 60], [47, 28, 11, 38, 56, 4, 27, 53], [34, 58, 0, 23, 17, 63, 57, 59], [15, 7, 43, 1, 33, 24, 18, 44], [51, 42, 50, 29, 8, 45, 2, 21], [54, 14, 20, 13, 61, 19, 6, 49], [9, 62, 22, 5, 46, 12, 55, 48], [37, 30, 36, 10, 39, 41, 26, 32]]
Table 1 Happiness = 400.0
Table 2 Happiness = 300.0
Table 3 Happiness = 1700.0
Table 4 Happiness = 1100.0
Table 5 Happiness = 2300.0
Table 6 Happiness = 1600.0
Table 7 Happiness = 400.0
Table 8 Happiness = 800.0
Total Fitness: 8600.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### 3.1. Fitness Proportionate Selection

In a typical **fitness proportionate selection** (also known as **roulette wheel selection**), individuals are selected based on their fitness values, with higher fitness values having a greater chance of being chosen. In our optimization problem, however, the fitness values can range from **negative** (indicating conflicts) to **positive** (indicating good relationships between guests).

To adapt the selection process to handle **negative fitness values**, we introduce the following modifications:

1. **Handling Negative Fitness**:
   - Negative fitness values (e.g., -1000) represent undesirable relationships between guests and are valid in our case.
   - However, to ensure that these negative values don't completely block a solution from being selected, we **normalize** the fitness values by adding an **offset** to make all fitness values non-negative.
   - The offset is the **absolute value of the minimum fitness** if it's negative, ensuring that all fitness values become positive or zero without altering the relative fitness ranking of solutions.

2. **Fitness Normalization**:
   - For each individual, we add the **offset** (if necessary) to their fitness value to ensure it's non-negative.
   - This ensures that individuals with negative fitness still have a chance to be selected but are less likely to be chosen compared to those with higher (positive) fitness.

3. **Roulette Wheel Selection**:
   - After normalizing the fitness values, the selection process proceeds as usual, with higher fitness solutions having a higher probability of being selected.
   - By adding the offset, we make sure that **negative fitness values** don't result in zero or near-zero chances of selection, allowing them to contribute to the evolutionary process.

In [9]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    if maximization:
        fitness_values = []
        min_fitness = min(ind.fitness() for ind in population)  # Find the minimum fitness value
        
        # Add a constant offset to ensure no negative fitness values
        offset = abs(min_fitness) if min_fitness < 0 else 0
        
        # Add the offset to each fitness value
        for ind in population:
            fitness_value = ind.fitness() + offset
            fitness_values.append(fitness_value)
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    
    # For each individual, check if random number is inside the individual's "box"
    box_boundary = 0
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

In [10]:
population = [WSSolution() for _ in range(10)]

fitness_proportionate_selection_result = fitness_proportionate_selection(population, maximization=True)
print('Fitness Proportionate selection result:', fitness_proportionate_selection_result)

Fitness Proportionate selection result: [[54, 6, 15, 39, 5, 38, 53, 11], [23, 16, 17, 14, 33, 18, 42, 12], [35, 48, 56, 22, 31, 58, 61, 41], [60, 49, 37, 45, 63, 46, 21, 26], [62, 4, 50, 1, 30, 9, 34, 7], [10, 47, 44, 28, 0, 36, 59, 8], [40, 52, 13, 32, 43, 25, 29, 57], [55, 20, 2, 51, 24, 19, 27, 3]]


#### 3.2. Ranking Selection

With Ranking Selection, instead of selecting based on raw fitness values, individuals are ranked by their fitness, and higher-ranked individuals (better seating arrangements) have a higher chance of being selected. This method ensures that negative fitness values aren´t automatically excluded, as selection is based on relative rank rather than absolute fitness. It maintains diversity by giving lower-ranked individuals a chance to contribute, preventing premature convergence.

In [11]:
def ranking_selection(population, maximization=False):
    n_parents = 1  # Always select one parent per call in the GA

    # Sort population based on fitness
    sorted_pop = sorted(
        population,
        key=lambda sol: sol.fitness(),
        reverse=maximization  # reverse=True if maximizing
    )

    # Assign linear ranks
    n = len(sorted_pop)
    ranks = np.arange(1, n + 1)
    total_rank = np.sum(ranks)
    probabilities = ranks / total_rank

    # If maximizing, best is first → reverse probabilities so high rank = high prob
    if maximization:
        probabilities = probabilities[::-1]

    selected = np.random.choice(sorted_pop, size=n_parents, p=probabilities, replace=True)
    return selected[0]  # Return single selected parent


In [12]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")


Fitness of individuals in population:
Individual 0: Fitness = 13200.00
Individual 1: Fitness = 8800.00
Individual 2: Fitness = 9500.00
Individual 3: Fitness = 15200.00
Individual 4: Fitness = 18500.00
Individual 5: Fitness = 11600.00
Individual 6: Fitness = 9500.00
Individual 7: Fitness = 13500.00
Individual 8: Fitness = 17100.00
Individual 9: Fitness = 17700.00

Ranking selection result:
Selected 1: [[1, 0, 58, 20, 59, 9, 22, 33], [54, 39, 14, 13, 44, 4, 2, 12], [60, 7, 31, 11, 3, 41, 5, 17], [52, 30, 43, 34, 50, 32, 51, 15], [35, 16, 6, 53, 38, 47, 26, 19], [42, 28, 40, 46, 62, 36, 49, 8], [56, 29, 37, 21, 48, 24, 61, 57], [18, 23, 45, 25, 63, 55, 10, 27]], Fitness = 18500.00
Selected 2: [[12, 2, 0, 5, 4, 49, 58, 48], [39, 36, 42, 30, 26, 52, 31, 11], [47, 10, 29, 18, 63, 55, 20, 21], [62, 38, 23, 33, 50, 32, 13, 3], [24, 25, 17, 15, 57, 45, 61, 46], [54, 28, 53, 51, 8, 37, 44, 7], [35, 19, 9, 27, 1, 6, 22, 60], [41, 56, 59, 34, 16, 43, 14, 40]], Fitness = 9500.00


In this case, we can see that the same individual (2nd best in rank) was selected twice. This is okay and makes that it coud happen especially with the high rank and small population.

In [13]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")

Fitness of individuals in population:
Individual 0: Fitness = 14600.00
Individual 1: Fitness = 17200.00
Individual 2: Fitness = 9100.00
Individual 3: Fitness = 15800.00
Individual 4: Fitness = 12900.00
Individual 5: Fitness = 10700.00
Individual 6: Fitness = 11500.00
Individual 7: Fitness = 10900.00
Individual 8: Fitness = 12200.00
Individual 9: Fitness = 9100.00

Ranking selection result:
Selected 1: [[17, 55, 51, 45, 49, 42, 58, 24], [43, 33, 53, 25, 36, 13, 30, 37], [48, 3, 1, 26, 16, 23, 20, 44], [40, 63, 41, 34, 0, 38, 21, 28], [47, 32, 7, 31, 57, 9, 54, 59], [61, 4, 11, 27, 14, 8, 19, 50], [46, 56, 5, 60, 22, 12, 6, 62], [39, 2, 15, 35, 52, 18, 10, 29]], Fitness = 11500.00
Selected 2: [[6, 23, 17, 43, 61, 48, 34, 58], [37, 1, 44, 11, 49, 56, 62, 47], [15, 52, 26, 24, 7, 8, 33, 59], [13, 2, 55, 36, 5, 22, 63, 12], [57, 35, 39, 20, 30, 38, 18, 60], [9, 27, 42, 46, 41, 25, 51, 21], [4, 50, 32, 53, 16, 54, 19, 31], [3, 45, 14, 40, 10, 29, 28, 0]], Fitness = 15800.00


#### 3.3. Tournament Selection

The tournament selection randomly chooses 𝑘 individuals at random from the population, compares their fitnesses, and returns the best individual out of those to be a parent. 

In [14]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

In [15]:
population = [WSSolution() for _ in range(10)]

tournament_selection(population)

[[25, 63, 35, 14, 62, 19, 41, 4], [9, 48, 31, 30, 26, 43, 53, 6], [40, 5, 16, 24, 2, 36, 1, 58], [50, 47, 27, 29, 12, 28, 39, 45], [59, 13, 3, 54, 23, 44, 8, 57], [42, 10, 49, 38, 55, 32, 37, 46], [34, 56, 17, 0, 22, 33, 61, 21], [7, 11, 60, 20, 52, 51, 18, 15]]

To make sure that our tournament_selection implementation truly picks the fittest individual, we wrapped it in a helper function, tournament_verification, which prints out every step of the process. This function is needed because the core algorithm samples competitors internally, not allowing us to check the candidates considered or their fitness except for the winner that is returned. In tournament_verification, we set 
k=10 and explicitly list all ten randomly chosen competitors along with their fitness values, then identify and print the one selected as the fittest by tournament_selection, allowing us to directly compare that result against the ten and confirm it truly has the highest fitness. This gives us clear, reproducible evidence that the tournament selection returns the strongest contender.

In [16]:
def tournament_verification(population, k=10):
    competitors = random.sample(population, k)
    
    print("Fitness of competitors:")
    for i in range(k):
        print(f"Competitor {i+1}: Fitness = {competitors[i].fitness()}")
    
    best_idx, best_competitor = max(enumerate(competitors),key=lambda pair: pair[1].fitness())
    
    print("\nTournament selection result:")
    print(f"Competitor {best_idx+1}: Fitness = {best_competitor.fitness():.2f}")

In [17]:
population = [WSSolution() for _ in range(50)]

tournament_verification(population)

Fitness of competitors:
Competitor 1: Fitness = 11300.0
Competitor 2: Fitness = 10800.0
Competitor 3: Fitness = 10000.0
Competitor 4: Fitness = 8300.0
Competitor 5: Fitness = 15100.0
Competitor 6: Fitness = 7100.0
Competitor 7: Fitness = 16800.0
Competitor 8: Fitness = 15200.0
Competitor 9: Fitness = 7600.0
Competitor 10: Fitness = 10100.0

Tournament selection result:
Competitor 7: Fitness = 16800.00


---

### <span style="color:#96d9f0;">4. Crossover Operators</span>

#### 4.1. Custom Crossover Operator (`table_level_crossover`)

In this section, we implement a custom crossover operator tailored to the constraints of our optimization problem. Unlike standard one-point crossover, which is suitable for flat representations, our solution requires a structured approach:

- Each solution is a nested list representing 8 tables with 8 unique guests each.
- Every guest (from 0 to 63) must appear exactly once across all tables.
- A direct crossover risks creating invalid offspring (with duplicated or missing guests).

To address this, our operator performs table-level crossover:

1. A one-point crossover is applied between the two parent solutions at the table level. This can result in invalid intermediate offspring that contain duplicate guests and omit others, violating the uniqueness constraint.

2. The resulting child is then repaired by:

    - Detecting duplicate and missing guests
    - Replacing duplicates with missing ones to restore a valid configuration
    - This ensures all offspring are valid solutions and suitable for use in the genetic algorithm.

In [18]:
def table_level_crossover(parent1, parent2):
    # Deep copies of the parent representations
    p1_repr = deepcopy(parent1.repr)
    p2_repr = deepcopy(parent2.repr)

    # Flatten guest lists
    total_guests = len(parent1.relationship_matrix)

    # One-point crossover at table level
    xo_point = random.randint(1, len(p1_repr) - 1)
    child1_repr = p1_repr[:xo_point] + p2_repr[xo_point:]
    child2_repr = p2_repr[:xo_point] + p1_repr[xo_point:]

    def repair(child_repr):
        # Count how many times each guest appears
        guest_counts = {}
        for table in child_repr:
            for guest in table:
                guest_counts[guest] = guest_counts.get(guest, 0) + 1

        # Identify missing and extra guests
        missing = [g for g in range(total_guests) if guest_counts.get(g, 0) == 0]
        extras = [g for g, count in guest_counts.items() if count > 1 for _ in range(count - 1)]

        # Replace extra guests with missing ones
        for table in child_repr:
            for i in range(len(table)):
                guest = table[i]
                if guest_counts[guest] > 1:
                    new_guest = missing.pop()
                    guest_counts[guest] -= 1
                    table[i] = new_guest
                    guest_counts[new_guest] = 1  # added to the table

        return child_repr

    # Repair both children
    repaired1 = repair(child1_repr)
    repaired2 = repair(child2_repr)

    return WSSolution(repr=repaired1), WSSolution(repr=repaired2)

In [19]:
solution1 = WSSolution()
solution2 = WSSolution()

table_level_crossover_result = table_level_crossover(solution1, solution2)

print('Parent 1:', solution1)
print('Parent 2:', solution2)
print('Offspring 1:', table_level_crossover_result[0])
print('Offspring 2:', table_level_crossover_result[1])

Parent 1: [[57, 45, 26, 11, 30, 33, 63, 6], [41, 56, 16, 10, 48, 18, 7, 31], [47, 59, 38, 49, 3, 20, 25, 19], [50, 55, 43, 51, 44, 8, 58, 5], [27, 2, 60, 23, 54, 32, 37, 13], [14, 15, 42, 17, 12, 28, 52, 9], [62, 4, 36, 46, 1, 24, 39, 61], [34, 35, 53, 0, 21, 22, 29, 40]]
Parent 2: [[29, 38, 54, 56, 51, 26, 43, 28], [60, 41, 62, 17, 49, 47, 22, 58], [2, 4, 48, 31, 10, 18, 30, 0], [37, 19, 36, 40, 16, 20, 33, 39], [15, 1, 32, 35, 61, 5, 21, 27], [12, 52, 7, 57, 14, 63, 59, 3], [11, 8, 45, 9, 44, 34, 46, 23], [24, 53, 55, 13, 25, 42, 50, 6]]
Offspring 1: [[62, 60, 26, 58, 30, 54, 51, 43], [41, 56, 29, 10, 48, 18, 28, 31], [47, 22, 38, 49, 17, 4, 2, 0], [37, 19, 36, 40, 16, 20, 33, 39], [15, 1, 32, 35, 61, 5, 21, 27], [12, 52, 7, 57, 14, 63, 59, 3], [11, 8, 45, 9, 44, 34, 46, 23], [24, 53, 55, 13, 25, 42, 50, 6]]
Offspring 2: [[63, 38, 59, 56, 57, 26, 45, 33], [25, 41, 20, 19, 49, 47, 16, 11], [7, 6, 48, 31, 10, 18, 30, 3], [50, 55, 43, 51, 44, 8, 58, 5], [27, 2, 60, 23, 54, 32, 37, 13], 

In [20]:
# Create 2 solutions
parent1 = WSSolution(nr_of_tables=8)
parent2 = WSSolution(nr_of_tables=8)

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = table_level_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[60, 33, 42, 56, 4, 6, 58, 1], [30, 52, 32, 25, 23, 15, 45, 63], [43, 10, 40, 22, 28, 26, 53, 29], [18, 7, 49, 3, 39, 57, 20, 11], [59, 14, 46, 55, 9, 17, 61, 62], [31, 48, 13, 34, 24, 38, 21, 47], [54, 37, 19, 44, 41, 35, 8, 27], [5, 50, 16, 36, 2, 51, 12, 0]]
Parent 2 Seating Arrangement:
[[33, 26, 23, 7, 58, 11, 41, 36], [9, 18, 62, 16, 57, 52, 21, 12], [15, 1, 30, 32, 47, 31, 48, 5], [28, 4, 14, 24, 13, 6, 0, 35], [38, 46, 22, 54, 39, 50, 55, 25], [17, 49, 34, 29, 27, 10, 59, 63], [3, 37, 20, 19, 44, 42, 45, 56], [2, 51, 43, 8, 60, 53, 40, 61]]

Offspring 1 Seating Arrangement:
[[50, 33, 42, 56, 4, 6, 58, 1], [30, 52, 32, 25, 23, 15, 45, 63], [36, 10, 16, 22, 28, 26, 12, 29], [18, 7, 49, 3, 39, 57, 20, 11], [59, 14, 46, 55, 9, 17, 5, 62], [31, 48, 13, 34, 24, 38, 21, 47], [54, 37, 19, 44, 41, 35, 0, 27], [2, 51, 43, 8, 60, 53, 40, 61]]
Offspring 2 Seating Arrangement:
[[33, 26, 23, 7, 58, 11, 41, 61], [9, 18, 62, 60, 57, 52, 21, 53], [15, 1, 30, 32, 4

In [21]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.2. Partially Matched Crossover

In [22]:
def partially_matched_crossover(parent1, parent2):
    # Flatten parent representations (PMC works on 1D permutations)
    def flatten(tables):
        return [guest for table in tables for guest in table]
    
    # Function to unflatten the list back to the original representation, after crossover
    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    table_size = parent1.table_capacity
    total_guests = len(parent1.relationship_matrix)

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(total_guests), 2))

    def pmc(parent1, parent2):
        child = [None] * total_guests
        # Copy crossover segment from parent2
        child[point1:point2] = parent2[point1:point2]
        
        # Set to track already placed guests in child
        placed_guests = set(child[point1:point2])

        # Mapping for conflict resolution
        for i in range(point1, point2):
            if parent1[i] not in placed_guests:
                val = parent1[i]
                pos = i
                # Resolve conflict by finding non-conflicting guests
                while parent2[pos] in placed_guests:
                    pos = parent1.index(parent2[pos])
                child[pos] = val
                placed_guests.add(val)

        # Fill remaining positions with non-conflicting genes
        for i in range(total_guests):
            if child[i] is None:
                # If the position is still None, place the corresponding guest from parent1
                if parent1[i] not in placed_guests:
                    child[i] = parent1[i]
                    placed_guests.add(parent1[i])

        return child

The Partially Matched Crossover (pmc) is a genetic algorithm operator designed for permutations, like our guest seating arrangement. It swaps a segment between two parent solutions and builds offspring by preserving the order and position of elements while avoiding duplicates. In our implementation, we adapted PMX to work with the wedding seating format by flattening table arrangements into 1D lists, applying PMX, and then converting them back. We carefully resolved conflicts during crossover by mapping duplicate entries to valid unused guests, ensuring that each guest appears exactly once in the final seating—maintaining valid, non-repetitive arrangements.

In [23]:
def partially_matched_crossover(parent1, parent2):
    """
    PMX crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    import random

    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def pmc(p1, p2):
        child = [None] * size
        # Copy the crossover segment from p1
        child[point1:point2] = p1[point1:point2]

        for i in range(point1, point2):
            if p2[i] not in child:
                val = p2[i]
                idx = i
                while True:
                    mapped = p1[idx]
                    idx = p2.index(mapped)
                    if child[idx] is None:
                        child[idx] = val
                        break

        # Fill remaining positions from p2
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]

        return child

    child1_flat = pmc(parent1_flat, parent2_flat)
    child2_flat = pmc(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2


In [24]:
# Create 2 solutions
parent1 = WSSolution(nr_of_tables=8)
parent2 = WSSolution(nr_of_tables=8)

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = partially_matched_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[51, 21, 8, 19, 28, 49, 2, 7], [16, 9, 30, 42, 45, 44, 48, 12], [10, 5, 46, 34, 3, 63, 23, 27], [15, 52, 26, 40, 4, 53, 13, 25], [32, 41, 37, 43, 6, 38, 31, 59], [35, 20, 54, 1, 0, 39, 62, 11], [58, 60, 22, 14, 36, 24, 18, 57], [56, 33, 17, 61, 29, 55, 50, 47]]
Parent 2 Seating Arrangement:
[[1, 12, 11, 38, 44, 8, 62, 52], [21, 28, 4, 27, 13, 40, 41, 10], [34, 5, 50, 37, 14, 16, 29, 25], [30, 55, 59, 60, 58, 43, 63, 31], [2, 9, 3, 36, 61, 0, 20, 32], [7, 35, 49, 47, 45, 42, 46, 39], [51, 19, 33, 26, 17, 24, 56, 15], [48, 23, 18, 53, 6, 54, 57, 22]]

Offspring 1 Seating Arrangement:
[[47, 12, 11, 0, 44, 8, 62, 52], [21, 28, 4, 27, 13, 40, 41, 10], [34, 5, 50, 37, 14, 16, 29, 25], [30, 55, 32, 60, 58, 43, 63, 7], [2, 9, 3, 36, 61, 38, 31, 59], [35, 20, 54, 1, 45, 42, 46, 39], [51, 19, 33, 26, 17, 24, 56, 15], [48, 23, 18, 53, 6, 49, 57, 22]]
Offspring 2 Seating Arrangement:
[[51, 21, 8, 19, 28, 54, 2, 31], [16, 9, 30, 42, 45, 44, 48, 12], [10, 5, 46, 34, 3,

In [25]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.3. Order Crossover

The Order Crossover (OX) is a genetic algorithm operator also designed for permutation problems, such as our guest seating arrangements. It selects a contiguous block of seats from one parent and preserves both the relative order and positional integrity of the remaining guests by filling in, in sequence, from the other parent’s list and skipping any guest already included. In our implementation, we adapt OX to the wedding format by flattening each table layout into a single list, choosing two crossover points, copying the segment from Parent 1 into Offspring 1 (and vice versa for Offspring 2), then rotating through the other parent’s flattened list to fill the empty slots while avoiding duplicates. Finally, we reshape the 1D offspring lists back into the original table structure, guaranteeing a complete, non-repetitive seating plan. 

In [26]:
def order_crossover(parent1, parent2):
    """
    Order crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def order(p1, p2):
        child = [None] * size
        child[point1:point2] = p1[point1:point2]
        
        for i in range(point2-size, point1):
            val = p2[i]
            idx = i
            while val in child:
                idx+=1
                val = p2[idx]
            child[i] = val

        return child

    child1_flat = order(parent1_flat, parent2_flat)
    child2_flat = order(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2

            
        


In [27]:
# Create 2 solutions
parent1 = WSSolution(nr_of_tables=8)
parent2 = WSSolution(nr_of_tables=8)

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply Order crossover
offspring1, offspring2 = order_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())

Parent 1 Seating Arrangement:
[[31, 20, 6, 15, 2, 57, 13, 11], [28, 7, 19, 54, 51, 35, 53, 47], [4, 49, 8, 26, 21, 32, 50, 23], [62, 25, 14, 42, 44, 48, 40, 18], [38, 41, 55, 9, 45, 58, 37, 3], [12, 5, 33, 39, 60, 16, 1, 56], [22, 43, 30, 24, 10, 0, 34, 46], [61, 17, 27, 63, 52, 59, 36, 29]]
Parent 2 Seating Arrangement:
[[15, 46, 9, 8, 26, 0, 53, 4], [59, 32, 57, 27, 44, 23, 54, 13], [47, 61, 41, 31, 3, 42, 49, 38], [50, 55, 2, 56, 21, 1, 39, 18], [17, 16, 52, 11, 10, 6, 19, 62], [33, 51, 40, 7, 45, 48, 34, 35], [60, 43, 29, 20, 28, 58, 5, 30], [37, 24, 22, 36, 14, 25, 63, 12]]

Offspring 1 Seating Arrangement:
[[29, 20, 6, 15, 2, 57, 13, 11], [28, 7, 19, 54, 51, 35, 53, 47], [4, 49, 8, 26, 21, 32, 50, 23], [62, 25, 14, 42, 44, 48, 40, 18], [38, 41, 55, 9, 45, 58, 37, 3], [12, 5, 33, 39, 60, 16, 1, 56], [22, 43, 30, 24, 10, 0, 34, 46], [61, 17, 27, 63, 52, 59, 36, 31]]
Offspring 2 Seating Arrangement:
[[12, 46, 9, 8, 26, 0, 53, 4], [59, 32, 57, 27, 44, 23, 54, 13], [47, 61, 41, 31, 3,

In [28]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


---

### <span style="color:#96d9f0;">5. Mutation Operators</span>

#### 5.1. Swap Mutation

The swap mutation operator introduces random changes to the seating arrangement by swapping two guests between different tables.

It's important to note that swapping two guests within the same table would not create any new configuration - it would simply result in the same solution as before. Since we want to introduce meaningful changes, the swap mutation ensures that the two guests are always swapped between different tables. This prevents the mutation from generating an invalid or identical solution.

This mutation helps the genetic algorithm explore new seating arrangements by shuffling guests across tables while respecting the overall constraints of the problem.

In [ ]:
def swap_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    if len(tables) < 2:
        return new_solution

    # Pick two random tables
    t1, t2 = random.sample(range(len(tables)), 2)
    if not tables[t1] or not tables[t2]:
        return new_solution

    # Swap one person from each
    i1 = random.randint(0, len(tables[t1]) - 1)
    i2 = random.randint(0, len(tables[t2]) - 1)
    tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

    return new_solution

In [ ]:
solution = WSSolution()

mutated_solution = swap_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

#### 5.2. Inversion Mutation

The inversion mutation operator introduces variation by selecting a random segment of the seating plan and reversing the order of guests within that segment.

Since the seating arrangement is represented as a list of tables, the entire structure is first flattened into a single list containing all guests. The inversion is then performed on a random sublist, effectively inverting a continuous sequence of guests. After the inversion, the list is reshaped back into the original table format.

This mutation allows the algorithm to explore new configurations by reordering guests while preserving the structure of the solution. To ensure validity, a repair function is applied after mutation to guarantee that all guests remain uniquely assigned with no duplicates or omissions.

In [ ]:
def inversion_mutation_seating_Afonso(individual: WSSolution, mut_prob):
    tables = individual.repr  # Access the 8x8 seating arrangement

    # Flatten the 8x8 seating into a list of 64 guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation
    mutated_flat = inversion_mutation(flat, mut_prob)

    # Reshape back to 8 tables of 8
    new_tables = [mutated_flat[i:i+8] for i in range(0, 64, 8)]

    # Return a new WSSolution with the mutated tables
    return WSSolution(repr=new_tables)

In [ ]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating_Afonso(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

In [ ]:
def inversion_mutation_seating_Rita(individual: WSSolution, mut_prob):
    # Access the seating arrangement (8 tables with guests)
    tables = individual.repr
    # Flatten the seating arrangement into a single list of guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation with the given probability
    if random.random() < mut_prob:
        # Select two random indices for the inversion
        start, end = sorted(random.sample(range(len(flat)), 2))
        # Reverse the segment between the two indices
        flat[start:end + 1] = reversed(flat[start:end + 1])

    # Reshape the flat list back into 8 tables of 8 guests each
    new_tables = [flat[i:i + individual.table_capacity] for i in range(0, len(flat), individual.table_capacity)]

    # Return a new WSSolution with the mutated seating arrangement
    return WSSolution(repr=new_tables, relationship_matrix=individual.relationship_matrix, nr_of_tables=individual.nr_of_tables)

In [ ]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating_Rita(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

#### 5.3. New Creative Mutation

In [ ]:
# Code (someone)

---

### <span style="color:#96d9f0;">6. Genetic Algorithm</span>

In [ ]:
def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    crossover_operator: Callable,
    mutation_operator: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        crossover_operator (Callable): Function to perform crossover between two individuals.
        mutation_operator (Callable): Function to mutate an individual.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found after evolving for max_gen generations.
    """
    population = initial_population

    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        new_population = []

        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))

        while len(new_population) < len(population):
            parent1 = selection_algorithm(population, maximization)
            parent2 = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected parents:\n{parent1}\n{parent2}')

            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                if verbose:
                    print('Crossover applied.')
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)
                if verbose:
                    print('Replication applied.')

            mutated1 = mutation_operator(offspring1, mut_prob)
            new_population.append(mutated1)
            if verbose:
                print(f'Mutated offspring 1: {mutated1}')

            if len(new_population) < len(population):
                mutated2 = mutation_operator(offspring2, mut_prob)
                new_population.append(mutated2)
                if verbose:
                    print(f'Mutated offspring 2: {mutated2}')

        population = new_population

        if verbose:
            best = get_best_ind(population, maximization)
            print(f'Best individual fitness this generation: {best.fitness()}')

    return get_best_ind(population, maximization)

In [ ]:
# Number of individuals in the initial population
population_size = 50

# Generate initial population of random WSSolutions
population = [WSSolution() for _ in range(population_size)]

In [ ]:
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best)
print("Fitness:", best.fitness())

In [ ]:
isa_try = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(isa_try)
print("Fitness:", isa_try.fitness())

In [ ]:
# Number of individuals in the initial population
population_size2 = 3

# Generate initial population of random WSSolutions
population2 = [WSSolution() for _ in range(population_size2)]

best2 = genetic_algorithm(
    initial_population=population2,
    max_gen=5,
    selection_algorithm=tournament_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=inversion_mutation_seating,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best2)
print("Fitness:", best2.fitness())

In [ ]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
pmc_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(pmc_test.repr)
print("Fitness:", pmc_test.fitness())

In [ ]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(pmc_test.repr))

In [ ]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
ranking_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(ranking_test.repr)
print("Fitness:", ranking_test.fitness())

In [ ]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(ranking_test.repr))